In [1]:
import pandas as pd
import os
import re
import numpy as np
from library import tables

In [2]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/descriptive/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data.csv'))
data = data[data.year == 'yr1718']

/Users/kylieleblancKylie/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# District Characteristics

In [3]:
print('There are', len(data), 'districts in Texas.')
print('Of these,', len(data[data.distischarter == 'Y']), 'are charters.')
print('Of traditional public schools only', len(data[data.distischarter == 'N'][data.eligible == 0]), 'are ineligible for DOI status.')
print(len(data[data.doi == True]), 'districts are districts of innovation.')

There are 1203 districts in Texas.
Of these, 180 are charters.
Of traditional public schools only 33 are ineligible for DOI status.
785 districts are districts of innovation.


/Users/kylieleblancKylie/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [14]:
data['type_description'].value_counts()

RURAL                            461
CHARTER SCHOOL DISTRICTS         180
NON-METROPOLITAN STABLE          174
OTHER CENTRAL CITY SUBURBAN      164
MAJOR SUBURBAN                    79
INDEPENDENT TOWN                  68
OTHER CENTRAL CITY                38
NON-METROPOLITAN FAST GROWING     28
MAJOR URBAN                       11
Name: type_description, dtype: int64

In [11]:
data.groupby(['geography'])['students_num'].mean()

geography
Rural         390.271150
Suburban    10329.209877
Town         2133.429630
Urban       36841.020408
Name: students_num, dtype: float64

In [5]:
data.groupby(['doi', 'distischarter', 'eligible'])['district'].count()

doi    distischarter  eligible
False  N              0.0          24
                      1.0         214
       Y              0.0         180
True   N              0.0           9
                      1.0         776
Name: district, dtype: int64

In [13]:
countXgeo = pd.DataFrame(data.groupby(['geography'])['district'].count())
countXgeo['proportion'] = countXgeo['district']/countXgeo['district'].sum()
countXgeo

,district,proportion
geography,,
Rural,461,0.450635
Suburban,243,0.237537
Town,270,0.263930
Urban,49,0.047898


In [16]:
countXtype = pd.DataFrame(data.groupby(['type_description'])['district'].count())
countXtype['proportion'] = countXtype['district']/countXtype['district'].sum()
countXtype

,district,proportion
type_description,,
CHARTER SCHOOL DISTRICTS,180,0.149626
INDEPENDENT TOWN,68,0.056525
MAJOR SUBURBAN,79,0.065669
MAJOR URBAN,11,0.009144
NON-METROPOLITAN FAST GROWING,28,0.023275
NON-METROPOLITAN STABLE,174,0.144638
OTHER CENTRAL CITY,38,0.031588
OTHER CENTRAL CITY SUBURBAN,164,0.136326
RURAL,461,0.383209


In [18]:
data[data.distname == "LEANDER ISD"].type_description

8485    MAJOR SUBURBAN
Name: type_description, dtype: object

# Student Characteristics

Total students

In [7]:
students_total = data['students_num'].sum()
print(students_total)

5343834


## By geography

In [8]:
geography_order = {'Urban': 1,
                   'Suburban': 2,
                   'Town': 3,
                   'Rural': 4}

In [9]:
students_geography = pd.DataFrame(data.groupby(['geography'])['students_num'].sum()).reset_index()
students_geography['Proportion'] = students_geography['students_num']/students_total
students_geography['order'] = students_geography['geography'].map(geography_order)
students_geography = students_geography.sort_values('order', axis = 0)
students_geography = students_geography.drop('order', axis = 1)
students_geography

,geography,students_num,Proportion
3,Urban,1805210,0.337812
1,Suburban,2509998,0.469700
2,Town,576026,0.107793
0,Rural,179915,0.033668


## By demographics

In [10]:
demographics = ['students_hisp_num', 'students_white_num', 'students_black_num']
number = []
proportion = []
for dem in demographics:
    number.append(data[dem].sum())
    proportion.append(data[dem].sum()/students_total)
students_demographics = pd.DataFrame(
                            {'Demographics': demographics,
                             'Number': number,
                             'Proportion': proportion
                            })
students_demographics

,Demographics,Number,Proportion
0,students_hisp_num,2802180,0.524376
1,students_white_num,1499559,0.280615
2,students_black_num,673291,0.125994


## By school

In [11]:
school_order = {'doi': 1,
                'tps': 2,
                'charter': 3}

In [12]:
students_schools = pd.DataFrame(data.groupby(['district_status'])['students_num'].sum()).reset_index()
students_schools['Proportion'] = students_schools['students_num']/students_total
students_schools['order'] = students_schools['district_status'].map(school_order)
students_schools = students_schools.sort_values('order', axis = 0)
students_schools = students_schools.drop('order', axis = 1)
students_schools

,district_status,students_num,Proportion
1,doi,3832308,0.717146
2,tps,1238841,0.231826
0,charter,272685,0.051028


# To Tables

In [13]:
dfs = [students_geography, students_demographics, students_schools]
rows = [4, 9, 13]
for df, row in zip(dfs,rows):
    tables.df_to_excel(file = table_path + 'Student Characteristics.xlsx',
                      df = df,
                      df_columns = ['Proportion'],
                      start_row = row,
                      start_col = 2)